In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('savant_data.csv')

In [4]:
data['velocity'].mean()

np.float64(87.4166420897409)

In [13]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.09744614316939955)

In [16]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
538     31.1     31.100
1822    21.7     21.697
534     24.1     24.100
2119    42.0     41.996
561     28.9     28.900
...      ...        ...
744     18.3     18.300
733     23.6     23.600
2647    31.9     31.900
4150    29.4     29.400
3819    35.0     35.000

[915 rows x 2 columns]


In [32]:
#mean_whiff = data['swing_miss_percent'].mean()
#data['Stuff+'] = (data['swing_miss_percent'] / mean_whiff) * 100

# Normalizing the columns
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].max()) * 100
data['xwOBA_norm'] = (1 - (data['xwoba'] / data['xwoba'].max())) * 100 # Inverse
data['RunExp_norm'] = (1 - (data['run_exp'] / data['run_exp'].max())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = 0.1, 0.6, 0.3

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
features[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()


,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,25.7,0.311,1.273859,123.867466
1,17.7,0.333,0.553341,94.339052
2,26.0,0.285,1.117130,114.082719
3,21.8,0.363,0.014441,77.232561
4,22.0,0.325,0.394366,89.255667


In [34]:
# Sort the DataFrame by the 'Stuff+' column in descending order (higher Stuff+ is better)
features_sorted = data.sort_values(by='Stuff+', ascending=False)
features_sorted.to_csv('savant_data.csv', index=False)